In [23]:
%matplotlib inline
from config import domainData
from config import num_classes as NUM_CLASSES
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
import torchvision
import utils
from torchvision import datasets, transforms
import numpy as np
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from alexnet import alexnet

In [24]:
net = models.vgg16(pretrained=True)
print(net)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    

In [25]:
net.classifier = nn.Sequential(*list(net.classifier.children())[:-1])
# print(net)

In [26]:
src = domainData['amazon']
tar = domainData['webcam']

In [27]:
src_transforms = transforms.Compose([
    transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.50,1.0), ratio=(1.,1.)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])    
])
tar_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
#     transforms.RandomResizedCrop(224, scale=(0.25,1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]) 
])

In [28]:
src_dataset = datasets.ImageFolder(src, transform=src_transforms)
tar_dataset = datasets.ImageFolder(tar, transform=tar_transforms)
srcSampler = torch.utils.data.sampler.RandomSampler(src_dataset)
tarSampler = torch.utils.data.sampler.RandomSampler(tar_dataset)
srcDataLen = len(src_dataset)
tarDataLen = len(tar_dataset)
use_gpu = True and torch.cuda.is_available()
gpu = 2

In [29]:
opt = {
    'src': 'Amazon',
    'tar': 'Webcam',
    'manual_seed':1,
    'batchSize':32,
    'use_gpu': use_gpu,
    'num_classes': 31,
    'epochs': 75,
    'momentum': 0.9,
    'lr': 2e-4,
    'lr_sch': 0,
    'lr_sch_gamma': 0.1,
    'p_lr_decay': 75,
    'n0': 1.,
    'alpha': 10,
    'beta': 0.75,
    'betas': (0.5,0.99),
    'net_wtDcy': 0.001,
    'btl_wtDcy': 0.001,
    'srcDataLen': srcDataLen,
    'tarDataLen': tarDataLen
}

In [30]:
torch.manual_seed(opt['manual_seed'])
if opt['use_gpu']: torch.cuda.manual_seed(opt['manual_seed'])

In [31]:
print("use_gpu: ", opt['use_gpu'])

use_gpu:  True


In [32]:
src_dataloader = torch.utils.data.DataLoader(src_dataset, batch_size=opt['batchSize'], 
                                             shuffle=(srcSampler is None), sampler=srcSampler,
                                            num_workers=2, pin_memory=False, drop_last=False)
tar_dataloader = torch.utils.data.DataLoader(tar_dataset, batch_size=opt['batchSize'],
                                            shuffle=(tarSampler is None), sampler=tarSampler,
                                            num_workers=2, pin_memory=False, drop_last=False)

In [33]:
def init_weights(m):
#     print(m)
    if isinstance(m, nn.Linear):
        init.xavier_normal(m.weight)
        init.constant(m.bias, 0.1)

In [34]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.classifier = nn.Sequential(nn.Linear(4096,256),
                                       nn.ReLU(inplace=True), nn.Linear(256, opt['num_classes']))
        self.classifier.apply(init_weights)
    
    def forward(self, x):
        x = self.classifier(x)
        return x

In [35]:
net2 = Model()

In [36]:
if opt['use_gpu']:
    net = net.cuda(gpu)
    net2 = net2.cuda(gpu)
    torch.backends.cudnn.enabled=True
    torch.backends.cudnn.benchmark=True

To computer norm of various parameters

```python
for name, param in net2.named_parameters():
    nrm = torch.norm(param, 2)
    zero = param.eq(0.).float().sum()
    nele = torch.numel(param)
    print(name, nrm.data[0], nele, zero.data[0])
```

In [37]:
criterion = nn.CrossEntropyLoss()
softmax = nn.Softmax(dim=1)

In [38]:
netfeatwt, netfeatbias = list(), list()
for name, param in net.features.named_parameters():
    if name in ['19.weight', '21.weight', '24.weight', '26.weight', '28.weight']: netfeatwt.append(param)
    elif name in ['19.bias', '21.bias', '24.bias', '26.bias', '28.bias']: netfeatbias.append(param)
    else: param.requires_grad=False

In [39]:
for name, param in net.named_parameters():
    print(name, param.requires_grad)

features.0.weight False
features.0.bias False
features.2.weight False
features.2.bias False
features.5.weight False
features.5.bias False
features.7.weight False
features.7.bias False
features.10.weight False
features.10.bias False
features.12.weight False
features.12.bias False
features.14.weight False
features.14.bias False
features.17.weight False
features.17.bias False
features.19.weight True
features.19.bias True
features.21.weight True
features.21.bias True
features.24.weight True
features.24.bias True
features.26.weight True
features.26.bias True
features.28.weight True
features.28.bias True
classifier.0.weight True
classifier.0.bias True
classifier.3.weight True
classifier.3.bias True


In [40]:
for name, param in net.classifier.named_parameters():
    if 'weight' in name: netfeatwt.append(param)
    elif 'bias' in name: netfeatbias.append(param)

In [41]:
net2_weight, net2_bias = list(), list()
for name, param in net2.named_parameters():
    if 'weight' in name: net2_weight.append(param)
    elif 'bias' in name: net2_bias.append(param)

In [42]:
sgd_params = [
    {'params': netfeatwt, 'lr': opt['lr'], 'momentum': opt['momentum'], 'weight_decay': opt['net_wtDcy'], 'lr_mul': 1., 'name': 'netfeatwt'},
    {'params': netfeatbias, 'lr': opt['lr'], 'momentum': opt['momentum'], 'weight_decay': 0*opt['net_wtDcy'], 'lr_mul': 2., 'name': 'netfeatbias'},
    {'params': net2_weight, 'lr': opt['lr'], 'momentum': opt['momentum'], 'weight_decay': opt['btl_wtDcy'], 'lr_mul': 10., 'name': 'net2wt'},
    {'params': net2_bias, 'lr': opt['lr'], 'momentum': opt['momentum'], 'weight_decay': 0*opt['btl_wtDcy'], 'lr_mul': 20., 'name': 'net2bias'}
]

# optimizer = optim.Adam(net2.parameters(), lr=opt['lr'], betas=opt['betas'], weight_decay=opt['weight_decay'])
# optimizer = optim.Adam(sgd_params, betas=opt['betas'])

optimizer_2 = optim.SGD(sgd_params)
# optimizer_2 = optim.SGD(net2.parameters(), lr=opt['lr'], momentum=opt['momentum'], weight_decay=opt['btl_wtDcy'])

lr_sch = None
if opt['lr_sch']=='step': lr_sch = optim.lr_scheduler.StepLR(optimizer, 20, opt['lr_sch_gamma'])
if opt['lr_sch']=='exponential': lr_sch = optim.lr_scheduler.ExponentialLR(optimizer_2, opt['lr_sch_gamma'])

In [43]:
def get_validation_acc(cm=False):
    tarData = iter(tar_dataloader)
    totalCorrects = 0.
    if cm: y_preds, y_true = list(), list()
    for tarimgs, tarlbls in tarData:
        tarimgs = tarimgs.cuda(gpu) if opt['use_gpu'] else tarimgs
        tarlbls = tarlbls.cuda(gpu) if opt['use_gpu'] else tarlbls      
        tarimgs = Variable(tarimgs, volatile=True)

        feat_ = net(tarimgs)
        logits = net2(feat_)

        _, preds = torch.max(softmax(logits).data, 1)
        totalCorrects += torch.eq(preds, tarlbls).float().sum()
        if cm: y_preds.append(preds.cpu().numpy()), y_true.append(tarlbls.cpu().numpy())
    valAcc = totalCorrects / opt['tarDataLen']
    if cm: return valAcc, y_preds, y_true
    return valAcc

In [44]:
p = np.linspace(float(1./opt['p_lr_decay']),1,opt['p_lr_decay'])

for epoch in range(opt['epochs']):
    srcData = iter(src_dataloader)
    totalCorrects = 0.
    totalClsLoss = 0.
#     experiment.log_current_epoch(epoch)
    
    n_p = opt['n0'] / pow((1. + opt['alpha'] * p[epoch]), (opt['beta']))
    print("n_p: ", n_p)
    for param_group in optimizer_2.param_groups:
        param_group['lr'] = opt['lr'] * param_group['lr_mul'] * n_p
        
    for srcimgs, srclbls in srcData:
        srcimgs = srcimgs.cuda(gpu) if opt['use_gpu'] else srcimgs
        srclbls = srclbls.cuda(gpu) if opt['use_gpu'] else srclbls
        srcimgs, srclbls = Variable(srcimgs), Variable(srclbls)
        
        feat_ = net(srcimgs)
        logits = net2(feat_)
        
        clsloss = criterion(logits, srclbls)
        totalClsLoss += clsloss.data[0] * opt['batchSize']
        
        _, preds = torch.max(softmax(logits).data, 1)
        totalCorrects += torch.eq(preds, srclbls.data).float().sum()
               
        optimizer_2.zero_grad()
        clsloss.backward()
        optimizer_2.step()
        
        if lr_sch: lr_sch.step()
        
    srcAcc = totalCorrects / opt['srcDataLen']
    srcLoss = totalClsLoss / opt['srcDataLen']
    valAcc = get_validation_acc()
#     experiment.log_metric("src_clsLoss", srcLoss, step=epoch)
#     experiment.log_metric("src_clsAcc", srcAcc, step=epoch)
#     experiment.log_metric('val_accuracy', valAcc, step=epoch)
#     experiment.log_epoch_end(epoch)
    print("Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, validation acc: {:.4f}".
          format(epoch+1, srcLoss, srcAcc, valAcc))

n_p:  0.910398960683
Epoch: 1, train loss: 1.9675, train acc: 0.4949, validation acc: 0.6088
n_p:  0.837535537332
Epoch: 2, train loss: 0.8508, train acc: 0.7632, validation acc: 0.5836
n_p:  0.776969504241
Epoch: 3, train loss: 0.6784, train acc: 0.8126, validation acc: 0.6239
n_p:  0.725725807496
Epoch: 4, train loss: 0.6252, train acc: 0.8222, validation acc: 0.6566
n_p:  0.68173161988
Epoch: 5, train loss: 0.5589, train acc: 0.8403, validation acc: 0.6226
n_p:  0.643495658493
Epoch: 6, train loss: 0.4698, train acc: 0.8608, validation acc: 0.6126
n_p:  0.609915886892
Epoch: 7, train loss: 0.4286, train acc: 0.8644, validation acc: 0.6390
n_p:  0.580159187126
Epoch: 8, train loss: 0.3632, train acc: 0.8885, validation acc: 0.6465
n_p:  0.55358331165
Epoch: 9, train loss: 0.3243, train acc: 0.9045, validation acc: 0.6591
n_p:  0.529684678724
Epoch: 10, train loss: 0.2921, train acc: 0.9084, validation acc: 0.6491
n_p:  0.508062512101
Epoch: 11, train loss: 0.2771, train acc: 0.9123, 

```
valAcc, y_preds, y_true = get_validation_acc(cm=True)

y_preds = [d for sublist in y_preds for d in sublist]
y_true = [d for sublist in y_true for d in sublist]

cm = confusion_matrix(y_true, y_preds)

np.set_printoptions(precision=2)
plt.figure(figsize=(20,20))
utils.plot_confusion_matrix(cm, classes=src_dataset.classes, normalize=False)
plt.show()
```